In [6]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
import os
import torch
from torchvision import datasets, models, transforms

pin_memory=False 



In [10]:
print(torch.cuda.is_available())
from torch.utils.data import Dataset
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(torch.cuda.get_device_name(cuda_id))


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#pil verison fix
import PIL

from PIL import __version__
print(__version__)


True
NVIDIA GeForce RTX 3070 Laptop GPU
cuda
9.0.1


In [11]:
import torchvision
torchvision.__version__  

'0.14.1+cu117'

In [16]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        self.image_paths = []
        self.labels = []

        for data_type in ['train', 'test']:
            for class_name in ['DME', 'DRUSEN']:
                class_path = os.path.join(self.root_dir, data_type, class_name)
                class_label = 1 if class_name == 'DME' else 0

                for image_name in os.listdir(class_path):
                    image_path = os.path.join(class_path, image_name)
                    self.image_paths.append(image_path)
                    self.labels.append(class_label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label


In [17]:
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
import torch.nn as nn
import torch.optim as optim
import os
import torch
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import models
from torchvision.transforms import transforms

# Define the image transformation pipeline
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Define the path to the data directories
data_dir = './Data/'
srgan_dir = './srgan-images/'

# Define the device to run the models on
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load model A and B
model_A = models.resnet50(pretrained=True)
model_A.fc = nn.Sequential(
    nn.Linear(2048, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 2)).to(device)
model_A.load_state_dict(torch.load('./models/model_A.pt', map_location=device))

model_B = models.resnet50(pretrained=True)
model_B.fc = nn.Sequential(
    nn.Linear(2048, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 2)).to(device)
model_B.load_state_dict(torch.load('./models/model_B.pt', map_location=device))

# Evaluate model A on the data in ./Data
y_true = []
y_pred = []
for class_name in ['DRUSEN', 'DME']:
    for split_name in ['train', 'test']:
        dataset = CustomDataset(os.path.join(data_dir, split_name, class_name), transform=transform)
        dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=os.cpu_count())
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            y_pred.append(model_A(images).argmax().item())
            if class_name == 'DRUSEN':
                y_true.append(0)
            else:
                y_true.append(1)

f1 = f1_score(y_true, y_pred)
acc = accuracy_score(y_true, y_pred)
auc = roc_auc_score(y_true, y_pred)

print("Model A evaluation results:")
print(f"F1 score: {f1}")
print(f"Accuracy: {acc}")
print(f"AUC: {auc}")

# Evaluate model B on the data in ./srgan-images
y_true = []
y_pred = []
for class_name in ['DRUSEN', 'DME']:
    for split_name in ['train', 'test']:
        dataset = CustomDataset(os.path.join(srgan_dir, split_name, class_name), transform=transform)
        dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=os.cpu_count())
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            y_pred.append(model_B(images).argmax().item())
            if class_name == 'DRUSEN':
                y_true.append(0)
            else:
                y_true.append(1)

f1 = f1_score(y_true, y_pred)
acc = accuracy_score(y_true, y_pred)
auc = roc_auc_score(y_true, y_pred)

print("Model B evaluation results:")
print(f"F1 score: {f1}")
print(f"Accuracy: {acc}")
print(f"AUC: {auc}")


FileNotFoundError: [Errno 2] No such file or directory: './Data/train/DRUSEN/train/DME'